In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import solve_triangular
rng = np.random.default_rng(8808206249012891)

## Sparse Gaussian Processes for Regression

$$ p(\mathbf{f}, \mathbf{u}) = 
        \mathcal{N}\left( \begin{bmatrix} \mathbf{f} \\ \mathbf{u} \end{bmatrix} \middle|
        \mathbf{0}, \begin{bmatrix} \mathbf{K_{\mathbf{f}\mathbf{f}}} & \mathbf{K_{\mathbf{f}\mathbf{u}}} \\ \mathbf{K_{\mathbf{u}\mathbf{f}}} & \mathbf{K_{\mathbf{u}\mathbf{u}}} \end{bmatrix} \right) $$

\begin{align}
    p( \mathbf{f}, \mathbf{u})
        &\approx q(\mathbf{f}, \mathbf{u}) \\
        &= p(\mathbf{f} | \mathbf{u}) q(\mathbf{u}) \\
        &= \mathcal{N}( \mathbf{f} | \mathbf{K}_{\mathbf{f}\mathbf{u}} \mathbf{K}_{\mathbf{u}\mathbf{u}}^{-1} \mathbf{u}, \mathbf{K}_{\mathbf{f}\mathbf{f}}^{-1} - \mathbf{K}_{\mathbf{f}\mathbf{u}} \mathbf{K}_{\mathbf{u}\mathbf{u}}^{-1} \mathbf{K}_{\mathbf{u}\mathbf{f}} )
            \mathcal{N}(\mathbf{u} | \boldsymbol{\mu}, \mathbf{S}) \\
\end{align}

### Lower Bound

\begin{align}
    \log (\mathbf{y})
        &= \log \int p(\mathbf{y} | \mathbf{f}) p(\mathbf{f} | \mathbf{u}) p(\mathbf{u}) \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} \\
        &\geq \int q(\mathbf{f}, \mathbf{u}) \log \frac{p(\mathbf{y} | \mathbf{f}) p(\mathbf{f} | \mathbf{u}) p(\mathbf{u})}{q(\mathbf{f}, \mathbf{u})} \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} \\
        &= \int q(\mathbf{f}, \mathbf{u}) \log \frac{p(\mathbf{y} | \mathbf{f}) p(\mathbf{f} | \mathbf{u}) p(\mathbf{u})}{p(\mathbf{f} | \mathbf{u}) q(\mathbf{u})} \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} \\
        &= \int p(\mathbf{f} | \mathbf{u}) q(\mathbf{u}) \log p(\mathbf{y} | \mathbf{f}) \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} + \int q(\mathbf{f}, \mathbf{u}) \log \frac{p(\mathbf{u})}{q(\mathbf{u})} \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} \\
        &= \int q(\mathbf{f}) \log p(\mathbf{y} | \mathbf{f}) \;\mathrm{d} \mathbf{f} + \int q(\mathbf{u}) \log \frac{p(\mathbf{u})}{q(\mathbf{u})} \;\mathrm{d} \mathbf{f} \;\mathrm{d} \mathbf{u} \\
        &= \mathbb{E}_{q(\mathbf{f})} \log p(\mathbf{y} | \mathbf{f})  - \mathbb{KL}(p(\mathbf{u}) \parallel q(\mathbf{u}))
\end{align}

In [ ]:
def rbf(x1, x2, variance=1.0, lengthscale=1.0):
    return variance * np.exp(-0.5 * ((x1 - x2.T) / lengthscale) ** 2)

In [ ]:
N = 100
X = np.sort(rng.uniform(-5, 5, N))[:, None]
K = rbf(X, X)
F = rng.multivariate_normal(np.zeros(N), K)
Y = rng.multivariate_normal(F, 0.02 * np.eye(N))[:, None]

In [ ]:
M = 4
tril_idx = np.tril_indices(M)
kernel_params = {"variance": widgets.FloatSlider(value=1, min=0.0001, max=3.0, step=0.01), "lengthscale": widgets.FloatSlider(value=1, min=0.0001, max=3.0, step=0.01)}
Z_wg = [widgets.FloatSlider(value=i, min=-5.0, max=5.0, step=0.1) for i in np.linspace(-5, 5, M)]
q_mu_wg = [widgets.FloatSlider(value=0, min=-5.0, max=5.0, step=0.1) for _ in range(M)]
q_sqrt_wg = [widgets.BoundedFloatText(value=0, min=-5.0, max=5.0, step=0.1, layout=Layout(width='50px')) for i in range(M*(M+1)//2)]
for i in np.cumsum(np.r_[0, 2:M + 1]):
    q_sqrt_wg[i].value = 1.0
args = {
    **kernel_params,
    **{"noise_variance": widgets.FloatSlider(value=1, min=0.0001, max=2.0, step=0.01)},
    **{f"Z[{i}]": v for i, v in enumerate(Z_wg)},
    **{f"q_mu[{i}]": v for i, v in enumerate(q_mu_wg)},
    **{f"q_sqrt[{i}]": v for i, v in enumerate(q_sqrt_wg)}
}
q_sqrt_full_wg = np.full((M, M), widgets.Label("0"))
q_sqrt_full_wg[tril_idx] = q_sqrt_wg
ui = widgets.VBox(
    [
        widgets.HBox([widgets.Label(r"$\sigma=$"), kernel_params["variance"], widgets.Label(r"$l=$"), kernel_params["lengthscale"],
                      widgets.Label(r"$\sigma_n=$"),args["noise_variance"]]),
        widgets.HBox([widgets.Label(r"$\mathbf{Z}=$"), widgets.VBox(Z_wg), widgets.Label(r"$\boldsymbol\mu=$"), widgets.VBox(q_mu_wg)]),
        widgets.HBox([widgets.Label(r"$\mathbf{L}=$"),
                      widgets.GridBox(np.ravel(q_sqrt_full_wg).tolist(), layout=Layout(grid_template_columns=f"repeat({M}, 50px)", justify_items="center")),
                     ])
    ],
    layout=Layout(display="flex", width="100%")
)

N_test = 1000
q_sqrt = np.zeros((M, M))
X_new = np.linspace(-5, 5, num=N_test)[:, None]

def plot_gp(**kwargs):
    plt.figure(num=2, figsize=(16, 8))
    plt.scatter(X, Y)
    
    # Variable Initialisation
    variance = kwargs["variance"]
    lengthscale = kwargs["lengthscale"]
    noise_variance = kwargs["noise_variance"]
    Z = np.array([kwargs[f"Z[{i}]"] for i in range(M)])
    order = np.argsort(Z)
    Z = Z[order, None]
    q_mu = np.array([kwargs[f"q_mu[{i}]"] for i in range(M)])[order, None]
    q_sqrt_flat = [kwargs[f"q_sqrt[{i}]"] for i in range(M*(M+1)//2)]
    q_sqrt[tril_idx] = q_sqrt_flat
    q_sqrt[order][:, order]
    q_diag = np.diag(q_sqrt)
#     print(q_sqrt @ q_sqrt.T)
    
    # Inference
    Kuu = rbf(Z, Z, variance=variance, lengthscale=lengthscale) + 1e-6 * np.eye(M)
    Kuf = rbf(Z, X_new, variance=variance, lengthscale=lengthscale)
    Kff = np.full(N_test, variance)
    Lu = np.linalg.cholesky(Kuu)
    A = solve_triangular(Lu, Kuf, lower=True)
    fvar = Kff - np.sum(A ** 2, 0)
    A = solve_triangular(Lu, A, trans=1, lower=True)
    fmean = A.T @ q_mu
    fvar += np.sum((q_sqrt.T @ A) ** 2, 0)
    Y_pred = fmean, fvar + noise_variance
    
    # Visualisation
    plt.errorbar(Z, q_mu, q_diag, fmt="none")
    plt.scatter(Z, q_mu)
    plt.plot(X_new, Y_pred[0])
    plt.fill_between(X_new.flat,
                     Y_pred[0].flat - 2 * np.sqrt(Y_pred[1].flat),
                     Y_pred[0].flat + 2 * np.sqrt(Y_pred[1].flat),
                     alpha=0.2)
    plt.xlim(-5.5, 5.5)
    plt.ylim(-5, 5)
    plt.show()
    
output = widgets.interactive_output(plot_gp, args)
widgets.VBox([output, ui])

In [ ]:
# import tensorflow as tf
# from gpflow.models import SVGP
# from gpflow.kernels import SquaredExponential as SE
# from gpflow.likelihoods import Gaussian

In [ ]:
# Z = np.linspace(-5, 5, M)[:, None]
# model = SVGP(SE(), Gaussian(), Z, whiten=False)
# model

In [ ]:
# objective = model.training_loss_closure((X, Y))
# print("ELBO:", objective().numpy())
# adam = tf.optimizers.Adam(0.02)
# maxiter = 500
# for i in range(maxiter):
#     adam.minimize(objective, model.trainable_variables)
#     if i % (maxiter // 10) == 0:
#         print(i, objective().numpy())

In [ ]:
# model

### Update Widgets

In [ ]:
# kernel_params["variance"].value = model.kernel.variance.numpy().item()
# kernel_params["lengthscale"].value = model.kernel.lengthscales.numpy().item()

# args["noise_variance"].value = model.likelihood.variance.numpy().item()

# for Zi, Zopt, q_mu_i, q_mu_opt in zip(Z_wg, model.inducing_variable.Z.numpy().flat, q_mu_wg, model.q_mu.numpy().flat):
#     Zi.value = Zopt
#     q_mu_i.value = q_mu_opt

# for q_sqrt_i, q_sqrt_opt in zip(q_sqrt_wg, model.q_sqrt.numpy()[0][tril_idx]):
#     q_sqrt_i.value = q_sqrt_opt